In [2]:
import plotly.figure_factory as ff
import json 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from pathlib import Path





class Leek:
    path_data = "D:\\Master\\S2\\TER\\GitLabTER\\leek-wars-generator-Lks-v3\\data"
    def __init__(self,dataOutcomeLeek):
        self.stats = dict()
        outcomeKeys = list(dataOutcomeLeek.keys())
        for enumFile in [enumFile for enumFile in os.listdir(Leek.path_data) if enumFile.startswith('enum')]:
            content = os.path.join(Leek.path_data,enumFile)
            f = open(content, "r")
            data=json.loads(f.read())
            filename = os.path.splitext(enumFile[len('enum'):])[0]
            self.stats.update({filename:[dict() for x in range(max(data["enumIndex"]))]})
            for i in range (len(data["enumIndex"])):
                enumName=data["enumName"][i].lower()
                if(enumName in outcomeKeys):
                    self.stats[filename][data["enumIndex"][i]].update({enumName:dataOutcomeLeek[enumName]})
                

def getLeekStats(filename_outcome_fight):          
    f = open(filename_outcome_fight)
    dataOutcome = json.load(f)
    dataOutcome=json.loads(dataOutcome)["fight"]["leeks"]
    leeks = []
    for idx_leek in range(len(dataOutcome)):
        leek_file = Leek(dataOutcome[idx_leek])
        leeks.append(leek_file)
    return leeks



#convertie un df string en dictionnaire dont la clef est titre de la colonne str_col_id et la valeur la serie de valeur de la colonne
def toSeries(df,lst_dct, str_col_id,filter_val="[0]"):
    grp = df[[str_col_id, "ID_ENTITY","ID_TURN"]].query(str_col_id+"!="+filter_val).groupby("ID_ENTITY")[str_col_id,"ID_TURN"]
    
    for key, values in grp:
        tmp_grp = grp.get_group(key)
        lst_tmp = [0 for i in range(tmp_grp[tmp_grp.columns[0]].size)]
        for id_row , val_row in  tmp_grp.iterrows() :
            lst_tmp[val_row["ID_TURN"]]=ast.literal_eval(val_row[str_col_id])
        if (not key in lst_dct):
            lst_dct.update({key:dict()})

        lst_dct[key].update({str_col_id:lst_tmp})
        
    return lst_dct



#
def getEvolutionStats(df , lst_statNegStat =[["1700","LIFE","" ,"LOST_LIFE"]]):

    lst_evolutionStat = []
    #liste de dictionnaire ayant pour clef les titres dans lst_statNegStat 
    dct=dict()
    for lst in lst_statNegStat:
        for str_col_id in lst[1:]:
            if( bool(str_col_id)):
                dct=toSeries(df,dct,str_col_id)
        lst_evolutionStat.append(dct)

    lst_life = dict()
    #pour tous les dictionnaires de lst_evolutionStat calcule la somme des valeurs associé à chaque colonne 
    for dct in lst_evolutionStat:
        for k in dct.keys(): 
            lst_life.update({k:{}})
            for z in range(len(lst_statNegStat)) :
                lst = lst_statNegStat[z]
                lst_life[k].update({lst[1]:[int(lst[0])]})
                lst=lst[1:]
                sum_lst = lst_life[k][lst[0]]
                for i in range(1,len(dct[k][lst[0]])+1):
                    sum_lst.append(sum_lst[i-1])
                    neg=1
                    for str_col_id in lst:
                            if( not(bool(str_col_id))):
                                neg= -1
                                continue
                            dct_val = dct[k][str_col_id]
                            sum_lst[i] += neg*sum(dct_val[i-1])
    return lst_life





#df.drop(columns=["Unnamed: 0"],inplace=True)

def recap_fight(df ,  filter_lst_val = ["MOVE_TO"] , id_startWith ='ID' , seq_endWith='s' ,  n_key = ["fight_gen_idLeek","dimension", "seq_lens","mean","std","min","max"]):


    id_cols = [col for col in df.columns if col.startswith(id_startWith)]
    feat_cols = [col for col in df.columns if not(col in filter_lst_val  ) and not(id_startWith in col)  and not col.lower().endswith(seq_endWith) ] 

    #print(feat_cols)
    df2 = df.copy()[id_cols+feat_cols]


    for i in range(len(df2[feat_cols[0]])):
        for c in feat_cols :
            df2[c][i] = pd.to_numeric(ast.literal_eval(df2[c][i]))



    to_column_map = df2.set_index(id_cols)

    grouped = df2.groupby(id_cols[:-1])

    dict_gen = dict()

    obj_json_stats = dict. fromkeys(n_key)

    #.append(dictionary, ignore_index=True)
    n_df = pd.DataFrame(columns=n_key)
    for name, group in grouped:


        dict_val = dict()
        for f in feat_cols :
            dict_val.update({f:group[f].apply(pd.Series).stack().values})

        treshold = 0
        stats = dict()

        for f in dict_val.keys() :
            tmp_obj = obj_json_stats.copy()
            seq_lens = [0]
            for e in dict_val[f]:
                if(e > treshold):
                    seq_lens[-1]+=(1/len(dict_val[f]))
                elif(seq_lens[-1]>0):
                    seq_lens.append(0)

            tmp_obj["seq_lens"] = sum(seq_lens)/len(seq_lens)
            tmp_obj["mean"] = df2[f].mean()[0]
            tmp_obj["std"] = df2[f].std()
            tmp_obj["min"] = df2[f].min()[0]
            tmp_obj["max"] = df2[f].max()[0]
            
            tmp_obj["dimension"]=f

            tmp_obj["fight_gen_idLeek"]=name
            n_df = n_df.append(tmp_obj, ignore_index=True)

    return n_df 





def m_flatten(l):
    return [item for sublist in l for item in sublist]


def getlst_statNegStat( leek,str_lst = ["LIFE"]):
    lst_statNegStat =[]
    for str_col_id in str_lst:
        kl = leek.stats 
        t_l = str_col_id.lower()
        for k, e in kl.items():
            dct= e[0]
            if(t_l in  dct):
                    t_l2=t_l.upper()
                    lst_statNegStat.append([str(dct[t_l]),t_l2,"","LOST_"+t_l2])
                    break
    return lst_statNegStat


def getlstValAndLstIdx( df, filename_test_json ):
    leeks=getLeekStats( filename_test_json )
    lst_statNegStat = getlst_statNegStat( leeks[0])  
    lst_life = getEvolutionStats(df,lst_statNegStat)
    return lst_life , lst_statNegStat



def mergeNegStat (lst_life ,cols ,id_="ID_LEEK") :

    df2  = pd.DataFrame(columns=[id_]+cols  )

    for k in lst_life.keys():
        tmp_obj=dict({id_:k})
        tmp_obj.update(lst_life[k])
        df2=pd.concat([df2,pd.DataFrame([tmp_obj])], ignore_index=True)

    df3= pd.DataFrame(columns=[id_]+cols  )
    for col in cols : 
        t= df2.set_index([id_])[col].apply(pd.Series).stack()
        df4=t.to_frame()
        df4.rename(columns={0:col},inplace=True)
        if(col == cols[0]):
            df3=df4
        else:
            df3=df3.join(df4)
        
    return df3

def getColsFromLstStatNegStat(lst_statNegStat):
    return m_flatten([ list(filter(lambda x: x , e[1:2] )) for e in lst_statNegStat ])


def printAxis(df5,_id="ID_LEEK"):
    key_leek = list(set([e[0] for e in df5.index.values]))
    for i,k in enumerate(key_leek):
        df_o = df5.iloc[df5.index.get_level_values(_id) == k].reset_index(drop=True)
        if(i==0):
            ax = df_o.plot()
        else:
            df_o.plot(ax=ax)
        ax.legend(key_leek)

    return ax



def getCumulStat(df,filename_test_json):
    lst_life , lst_statNegStat  = getlstValAndLstIdx(  df,filename_test_json )
    cols = getColsFromLstStatNegStat(lst_statNegStat)
    df5  = mergeNegStat (lst_life ,cols )
    return df5


def convert_XY_toCoord(XY):
    coef_x = XY//18
    coord_x = XY%18
    
    sign_x =  0 if coord_x < coef_x else -18
    print(sign_x)
    coef_y = XY//17
    coord_y= XY%17
    sign_y = 0 if coord_y < coef_y  else -18
    print(sign_y)
    
    tmp_x = None
    return (coord_y+sign_y,-(coord_x+sign_x))



def getCoords(df):
    dtr = dict()
    dtr = toSeries(df,dtr, "MAPS")
    coords = dict()
    for k in dtr.keys() :
        dtr_map = dtr[k]["MAPS"]
        coords.update({k:dict()})
        for i in range(len(dtr[k]["MAPS"])):
                if( len(dtr_map[i]) > 1 ):
                    coords[k].update({i:[convert_XY_toCoord(t_coord) for t_coord in dtr_map[i]]})
    return coords




def getMapPlot(df,coords):
    dict_tuple = dict()

    for k1 in coords.keys(): #pour chaque leek 
        dict_tuple.update({k1:[[],[]]})
        coords_1 = coords[k1]
        for k2 in coords_1.keys():#pour chaque tour 
            for coord in coords_1[k2]: # pour chaque position 
                for i , e in enumerate(coord): #pour chaque dimension de la position 
                    dict_tuple[k1][i].append(e)
    return dict_tuple

def dfFromCoordDict(dct, str_coord=["x","y"]):
    df_final = pd.DataFrame()
    print(dct)
    for k,v in dct.items():
        df = pd.DataFrame(columns=["ID_LEEK"]+str_coord)
        for i , l in enumerate(v) : 
            df[str_coord[i]] = l
        df["ID_LEEK"] = k
        df_final = pd.concat([df_final,df])
    return df_final.reset_index()


def plot2DMap(df,colors = ['red', 'green', 'blue', 'yellow', 'black', 'purple', 'orange', 'pink', 'brown', 'grey', 'olive', 'cyan']):
    grp = df.groupby(["ID_LEEK"])
    fig, ax1 = plt.subplots(figsize=(17,17))
    for i, k in enumerate(grp.groups.keys()):
        df_o = grp.get_group(k)
        ax1.scatter(df_o["x"], df_o["y"], c=colors[i % len(colors)], alpha=0.5, edgecolors='none', s=30)
    return ax1




import shutil

class SaveStat :
    
    
        
    def __init__(self,_filename_test_csv,_filename_test_json,outcome_filename):
        if(os.path.isfile(outcome_filename)):
            outcome_filename= Path(outcome_filename).stem

      

        self.filename = os.path.splitext(outcome_filename)[0]
        self.path_out = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\\data",self.filename).replace("\\","/")
        self.position_map_save = os.path.join(self.path_out,"mapPosition").replace("\\","/")
        self.stat_save = os.path.join(self.path_out,"stats").replace("\\","/")
        self.recap_save =  os.path.join(self.path_out,"recaps").replace("\\","/")
        
        self.filename_test_csv=_filename_test_csv
        self.filename_test_json = _filename_test_json


        self.init_dir()
        
        self.filename_evolutionCurve = "evolutionCurve.csv"
        self.filename_recap = "recap.csv"
        self.filename_position = "position.csv"
        


        self.init_dataframe()



        
        


    def init_dir (self):
        isExist = os.path.exists(self.path_out)

        isExist = os.path.exists(self.path_out)

        if isExist:
            shutil.rmtree(self.path_out)

        os.mkdir(self.path_out)    
        os.mkdir(self.position_map_save)
        os.mkdir(self.stat_save)
 
    


    def init_dataframe(self):
            self.df = pd.read_csv(self.filename_test_csv, sep=";")      
            self.df.drop(columns=["Unnamed: 0"],inplace=True)
            #display(self.df)
            
            
    def saveMap(self):
        coords = getCoords(self.df)
        dict_tuple = getMapPlot(self.df,coords)
        df78 = dfFromCoordDict(dict_tuple)
        print('ii')
        display(df78)
        ax1 = plot2DMap(df78)
        ax1.figure.savefig( os.path.join(self.position_map_save , self.filename+".png"))
        df78.to_csv( self.recap_save+self.filename_position, mode='a', sep=";")


    def saveStat(self):
        df5 = getCumulStat(self.df , self.filename_test_json)
        ax= printAxis(df5)
        df5.to_csv(self.recap_save+self.filename_evolutionCurve, mode='a', sep=";")
        ax.figure.savefig( os.path.join(self.stat_save , self.filename +".png"))



    def saveRecap(self):
        n_df = recap_fight(self.df)
        n_df.to_csv(self.recap_save+self.filename_recap, mode='a', sep=";")


    def writeFiles(self):
        self.saveMap()
        self.saveStat()
        self.saveRecap()




def _fucn(outcome_filename ):
	
	filename = os.path.splitext(outcome_filename)[0]


	path_out = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\\data",filename)
	filename_test_csv = os.path.join(path_out,filename+".csv")
	filename_test_json = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\outcome",filename+".json")




	SaveStat(filename_test_csv,filename_test_json,outcome_filename).writeFiles()

In [60]:


outcome_filename_2 = "150_489_12-62_1680156349326.json"

outcome_filename = "145_487_12-62_1680155935861.json"


filename = os.path.splitext(outcome_filename)[0]


path_out = os.path.join("..","data",filename).replace("\\","/")
filename_test_csv = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\\data","145_487_12-62_1680155935861.csv").replace("\\","/")
filename_test_json = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\outcome",outcome_filename).replace("\\","/")

filename_test_csv2 = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\\data","150_489_12-62_1680156349326.csv").replace("\\","/")
filename_test_json2 = os.path.join("D:\\Master\\S2\\TER\\GitLabTER\\AfterFight\outcome",outcome_filename_2).replace("\\","/")



In [53]:
import shutil
print(path_out)

../data/145_487_12-62_1680118885684


In [102]:

 def convert_XY_toCoord(XY):
    coef_x = XY//18
    coord_x = XY%18
    sign_x =  0 if coord_x < coef_x else -18
    coef_y = XY//17
    coord_y= XY%17
    sign_y = 0 if coord_y < coef_y  else -18
    
    tmp_x = None
    return (coord_y+sign_y,-(coord_x+sign_x))

In [105]:
convert_XY_toCoord(428)

(3, -14)

In [ ]:
print(SaveStat(filename_test_csv ,filename_test_json,outcome_filename).writeFiles())
print("iii")
print(SaveStat(filename_test_csv2 ,filename_test_json2,outcome_filename_2).writeFiles())

In [ ]:

import shutil
from subprocess import Popen, PIPE
import sys
from threading import Thread, Lock
import json
import random
import time
import requests
from math import ceil
from uuid import getnode
import socket 
import os 

import pandas as pd

import numpy as np 
from pathlib import Path
from filenameVerif import _emptyTuple,filename_isvalid,splitIds,getIdsPos,getGenPos,getFightPos
from ReaderFilesEager import *
import ast

file_csv = "D:\Master\S2\TER\GitLabTER\te\12-62_1681156232985.csv"
def groupTo_dct(group , str_col):
    lst_2={}

    
    for e in group[str_col]:
        print(e)
        if(bool(e)):
            if( not(isinstance(e[1] , int))):
                    lst=[]
                    for e3 in ast.literal_eval(e[1]).tolist():
                        lst+=ast.literal_eval(e3)
                    lst_2.update({e[0]:lst})
    return lst_2



df55 = pd.read_csv(file_csv,sep=";",encoding="utf-8")
group = df55.groupby(["ID_ENTITY"])
print(group)
lost_life  = group['LOST_LIFE'].sum().tolist()
life =  [e for e in group["LIFE"] ]
dct_1 = groupTo_dct(group,"LOST_LIFE")
dct_2 = groupTo_dct(group,"LIFE")

In [91]:
def isInInterval ( e, interval ):
    e = int(e)
    return e >= interval[0] and e< interval[1] 


def reversedict( in_dict ):
        res_dict = dict()
        for k ,v in in_dict.items() :
            res_dict.update({v : k})
        return  res_dict

def print_pyth(_str):
    print("[PYTH]" + str(_str))

def isJsonFile( filename ):
    return filename.endswith(".json")

def errorMessage( _class : object, _method :str ,  _str: str  ):
    print("[ERROR] " + (" from :" + type(_class ) +",") if _class !=""  else "" + ("method :" +_method +",") if _method !="" else "" +   " description : " + _str)

def errorMessage( _str: str ):
    errorMessage( "" , "", _str)

def errorMessage( _method :str, _str: str ):
    errorMessage( "" , _method, _str)


def errorMessagePyth( _str: str ):
    print_pyth(errorMessage( _str ))

def errorMessagePyth( _method:str , _str: str ):
    print_pyth(errorMessage(_method , _str ))

def errorMessagePyth( _class:str , _method:str , _str: str ):
    print_pyth(errorMessage(_class , _method , _str ))


def errorMessageFromClass(obj : object, _str: str ):
    errorMessage( obj ,"", _str)

def errorMessageFromClassPyth(obj:object , _str: str ):
   print_pyth(errorMessage( obj , "" , _str ))


def notFound():
    return -1

def notSetId():
    return -1

def badInsert():
    return -1

def outOfRange():
    return -1

def strColInCols(str_col:str , df ):
    return str_col in df.columns


In [217]:
import pandas as pd 
import json
from myutil  import *
import os 
import numpy as np


'''
ReadActions :
    - STATIC :
        - MAP_SIZE : nombre de cases sur la map
        - intervals_ends : fin des intervalles des types d'actions ( e.g Times : 100 ,  Buffs: 200 , ... )
        - intervals_dict : dictionnaire inversé de intervals_dict ( e.g { {START_FIGHT: 0 ,...,USE_WEAPON: 16  , LOST_PT : 100 , ... , NOVA_VITALITY :  112 , ... }  }# le fichier filename_dfActions loader 
        - intervals_dictRev : dictionnaire des types d'actions ( e.g {0:START_FIGHT ,...,16: USE_WEAPON , ... } # le fichier filename_dfActions loader reversed
        - filename_dfActions : nom du fichier json contenant les actions de chaque types d'actions ainsi que la taille des intervalles 
        - ex_idx : liste des index des membres du json  a exclure (e.g [0,2,3]<=> interval_size , Funs , Others )
        - lst_col : liste des colonnes non calculer (directement accessible via l'outcome ) à stocker dans le dataframe
        - id_col : liste des colonnes contenant les ids identifiants les lignes du dataframe
        - deg_col : liste des colonnes contenant les degats infligés ou subis par les entités (parmis les colonnes décritent dans lst_col)
        - audit_col : liste des types d'actions performer pendant le tour (e.g [0,0,1,2,1,1] <=> [df.cols[0] , df.cols[0], df.cols[1] , ...] <=> [Times , Times , Buffs , ...])

    - NON STATIC :
        - numGen : numéro de la génération
        - numFight : numéro du combat
        - beg_notCodeAct_col : index indiquant le début des colonnes qui ne sont pas des liste de codes d'actions
        - entities_ids : les ids réelle des entités (telle que spécifié dans le scénario["entities"][i]["id"] ) impliquées dans le fight du dataframe 
        - int_col_idx_in_lst_col : liste des index des colonnes contenant des ints dans lst_col 

'''
class ReadActions :
    MAP_SIZE = 612 
    intervals_ends = dict()
    intervals_dict = dict()
    intervals_dictRev = dict()
    filename_dfActions = os.path.join("..","..","leek-wars-generator-Lks-v3","data","df_Action.json")
    ex_idx = [0,2,3]
    lst_col = ['LOST_LIFE','LIFE','MOVE_TO','WEAPON_DEG','CHIP_DEG','MAPS','WEAPON_ID','CHIP_ID']
    id_col = ['ID_GEN','ID_FIGHT','ID_ENTITY','ID_TURN']
    deg_col = ['LOST_LIFE','LIFE','WEAPON_DEG','CHIP_DEG']
    audit_col = ["ACTIONS"]
    
    #maps = [ [0]*MAP_SIZE ,[0]*MAP_SIZE]
    
    def __init__(self,ids :list ,numGen :int , numFight:int ,path_atm :str ):
        self.numGen = numGen
        self. numFight = numFight
        self.numTurn = 0
        ReadActions.init_intervals(ReadActions.filename_dfActions)#,ReadActions.ex_idx)
        self.nb_type = len(ReadActions.intervals_ends)
        self.lst_col_code = [list(ReadActions.intervals_ends.keys())[i] for i in range(1,len(ReadActions.intervals_ends)) ] 
        self.df = pd.DataFrame(columns=self.lst_col_code+ReadActions.lst_col+ReadActions.id_col+ReadActions.audit_col)
        self.beg_notCodeAct_col = len(self.df.columns) - len(ReadActions.lst_col)-len(ReadActions.id_col)-1
        self.entities_ids = ids 
        self.setIntCol()
        
    def setDF(self,df2):
        if(df2.columns == self.df.columns):
            self.df = pd.concat([self.df , df2])
            
        
    def setIntCol(self):
        self.int_col_idx_in_lst_col = []
        for i,d in enumerate(self.df.columns.tolist()) :
            if d in ReadActions.deg_col :
                self.int_col_idx_in_lst_col+=[i]
        self.int_col_idx_in_lst_col.sort()


    @property
    def getNumGen(self):
        return self.numGen
    
    @property
    def getNumFight(self):
        return self.numFight
    
    @property
    def getNumTurn(self):
        return self.numTurn
    
    @property
    def getDF(self):
        return self.df
    
    @property
    def getEntitiesIds(self):
        return self.entities_ids
    



    @staticmethod
    def emptyIntervalsEnd():
            ReadActions.intervals_ends.update({'BEGIN':0})


    
    @staticmethod
    def init_intervals(filename_json_dfA :str ,ex_idx=[]):
            """
            filename_json_dfA : fichier json contenant les actions de chaque types d'actions ainsi que la taille des intervalles
            ex_idx : liste des index des membres du json  a exclure (e.g [0,2,3]<=> interval_size , Funs , Others )

            Description : initialise les attributs intervals_dict , intervals_dictRev , intervals_ends de la classe ReadActions grâce au fichier json passer en paramètre
            """
            if( isJsonFile( filename_json_dfA ) ):
                ReadActions.emptyIntervalsEnd()
                #Load json file
                with open(filename_json_dfA, 'r') as f:
                    df_actions_json = json.load(f)
                interval_size = df_actions_json["interval_size"]

                #ignore interval_size from dictionnary
                del df_actions_json["interval_size"]

                #ignore ex_idx from dictionnary
                for to_ignore in ex_idx :
                    del df_actions_json[to_ignore]

                #extract info from dict and load it in class attributes 

                idx_keyOf_action = 0
                idx_intcodeOf_action = 1
                for i,group_of_actions in enumerate(df_actions_json.items())  :
                    id_group,lst_actions=group_of_actions
                    
                    for action in lst_actions.items():
                        ReadActions.intervals_dict.update({action[idx_keyOf_action]:action[idx_intcodeOf_action]})

                    ReadActions.intervals_ends.update({id_group:(interval_size+1)*(i+1)})
                ReadActions.intervals_dictRev = reversedict(ReadActions.intervals_dict)
            else :
                errorMessagePyth("file"+filename_json_dfA+" must be a json file")
        
     
    
    @staticmethod
    def whileToken(str_token : str , lst_actions :list ):
        """
        str_token : le string correspondant au token a chercher dans lst_actions ( un token est exprimer sous la forme d'un int dans lst_actions)
        lst_actions : liste d'actions  a parcourir   ( lst_action[i][0] == code_token ) 

        Description : parcours lst_actions jusqu'a ce que le token soit trouvé ou que la fin de la liste soit atteinte 
        Return : l'index de l'action token dans lst_actions tel que ( lst_action[index][0] == code_token )  ou -1 si le token n'a pas été trouvé
        """
        for i in range(len(lst_actions)) :
            action = lst_actions[i]
            if(action[0] == ReadActions.intervals_dict.get(str_token)):
                i+=1
                return i
        return notFound()
    

    @staticmethod        
    def isEndFight(act_token: int):
        return act_token == ReadActions.intervals_dict.get("END_FIGHT")
    
    @staticmethod 
    def isBeginFight(act_token: int):
        return act_token == ReadActions.intervals_dict.get("START_FIGHT")
    
    @staticmethod 
    def isBeginTurn(act_token: int):
        return act_token == ReadActions.intervals_dict.get("NEW_TURN")
    
    @staticmethod 
    def isEndTurn(act_token: int):
        return ReadActions.isEndFight(act_token) or ReadActions.isBeginTurn(act_token ) or act_token == ReadActions.intervals_dict.get("END_TURN")
    

    @staticmethod
    def getDfLstTypeActions():
        return list([[]])
    
    @staticmethod
    def getDfLstAudit():
        return list([[]])
    
    @staticmethod
    def getDfLstInLstCol():
        return [0]
    
    @staticmethod
    def getDfLstNotInLstCol():
        return []
    

    def getNewRow(self):
        lst =[]

        #-----------------Type Actions [Times,Buffs , ... ] -----------------
        idx = 0 
        idx_end = self.beg_notCodeAct_col 
        #Type of actions , e.g Times,Buffs,Funs,Effects , Others 
        for i in range (idx,idx_end):
            lst.append(ReadActions.getDfLstTypeActions())

        t=0
        #----------------- Lst Col -----------------
        idx = idx_end
        idx_end +=len(ReadActions.lst_col)
        for i in range (idx , idx_end):
            if i in self.int_col_idx_in_lst_col[t:] :
                lst.append(ReadActions.getDfLstInLstCol())
                t+=1
            #elif ReadActions.lst_col[i-idx] in ["WEAPON_ID","CHIP_ID"]:
            #    lst.append(notSetId())
            else :
                lst.append(ReadActions.getDfLstNotInLstCol())

        #----------------- Not in Lst Col -----------------
        idx = idx_end
        idx_end += len(ReadActions.id_col)
        for i in range (idx ,  idx_end):
            lst.append(notSetId())

        #----------------- Audit -----------------
        idx = idx_end
        idx_end += len(ReadActions.audit_col)
        for i in range (idx,idx_end):
            lst.append(ReadActions.getDfLstAudit())#actions



        return lst
    
    def idxNextRow(self):
        return self.getCurrentIDX()+1
    
    def addNewRow(self):
        self.df.loc[self.idxNextRow()]=self.getNewRow()
    
        
    def addNewTurn(self,idx_turn: int):
        """
        idx_turn : index du tour a ajouter
        Description : ajoute une nouvelle ligne et set sont indexe de tour à idx_turn
        """
        self.addNewRow()
        #self.numTurn = idx_turn
        self.setColumn ('ID_TURN', idx_turn )
        
    def getCurrentIDX (self):
        return len(self.df)-1

    def setColumn ( self , str_col:str ,val):
        """
        Description : set la colonne str_col de la ligne courante à val
        """
        if ( strColInCols(str_col ,self.df)):
            self.df.loc[self.getCurrentIDX(),str_col]= val 
    

    def addToColumn(self,str_col:str,val,pos:int=0):
        """
        val : valeur a ajouter a la liste de la colonne str_col de la ligne courante
        pos : position de la valeur a ajouter dans la liste de la colonne str_col de la ligne courante (par defaut 0)

        Description : ajoute une valeur a la liste de la colonne str_col de la ligne courante
        """
        if (strColInCols(str_col ,self.df)):
            lst=self.df.loc[self.getCurrentIDX(),str_col]

            if( len(lst )< pos):
                errorMessagePyth(self,"addToColumn" , "index out of range")

            if( not(bool(lst)) or len(lst )== pos):
                lst+=[0]
            
            lst[pos]+= val  

        return pos 
            
            
    def addToColumnList(self,str_col:str,val,pos:int=0):
        """
        Attention : on embedde la valeur dans une liste car la colonne str_col est de type liste 
        """
        if(len(self.df[str_col]) == 0  or not(isinstance(self.df[str_col][0],list))):
            errorMessagePyth(self,"addToColumnList" , "column "+str_col+" must be a list")
            return badInsert()
        else :
            return self.addToColumn(str_col , [val],pos) 

    def extendColum(self,str_col:str,val:list):
        """
        Description : on ajoute une liste a la liste de la colonne str_col de la ligne courante
        """
        if (strColInCols(str_col ,self.df)):
            self.df.loc[self.getCurrentIDX(),str_col].extend(val)
       

    def addToAudit(self , idx:int , idx_of_actionType :int ,   act_token:int  ):
        """
        Description : on ajoute l'index du type de l'action courante dans la colonne audit correspondante
        """
        err_code=self.addToColumnList (  ReadActions.audit_col[idx],idx_of_actionType)
        if(err_code == badInsert()):
            errorMessagePyth(self, "addToCategory" , "bad insert")
            
    def addToGroupAction(self,idx_of_actionType:int , act_token:int):
        err_code = self.addToColumnList (list(ReadActions.intervals_ends.keys())[idx_of_actionType],act_token)
        if(err_code == badInsert()):
            errorMessagePyth(self, "addToCategory" , "bad insert")




    def setIDs(self,entity_id:int):
        """
        Description : set les colonnes IDs de la ligne courante sauf ID_TURN 
        """
        self.setColumn ('ID_GEN',self.numGen)
        self.setColumn ('ID_FIGHT',self. numFight)
        self.setColumn ('ID_ENTITY',self.getEntityID(entity_id))
        #self.setColumn ('ID_TURN',self.idx_turn)
        
    def getEntityID(self ,entity_id:int ):
        """
        Description : retourne l'ID de l'entité entity_id si elle existe sinon retourne outOfRange()
        """
        if len(self.entities_ids) > entity_id :
            return self.entities_ids[entity_id]
        
        else :
            errorMessagePyth(self,"getEntityID" , "index out of range")
            return outOfRange()
        
    @staticmethod
    def getIdxCodeToken():
        return 0
    @staticmethod
    def getIdxEntityToken():
        return 1
    
    @staticmethod
    def getCodeToken(_lst :list):
        return _lst[ReadActions.getIdxCodeToken()]
    @staticmethod
    def getEntityToken(_lst :list):
        return _lst[ReadActions.getIdxEntityToken()]
    
    def getTurn( self,lst_actions :list ,idx_turn :int ):
        entity_id = -1 
        
        # continue jusqu'au token LEEK_TURN
        nbActionRead=ReadActions.whileToken("LEEK_TURN",lst_actions)

        k = nbActionRead

        if(k==notFound()):
            return k , None 
        
        act_token = ReadActions.getCodeToken(lst_actions[k])
        entity_id =  ReadActions.getEntityToken(lst_actions[k-1])
        
        self.addNewTurn(idx_turn)
        self.setIDs(entity_id)

        #si le tour est vide : 
        if(self.isEndTurn(act_token)):
            return k , entity_id 
        
        idx=0
        while k  < len(lst_actions):

            act_token = ReadActions.getCodeToken(lst_actions[k])

            #si le tour est fini : 
            if(self.isEndTurn(act_token) ):
                return k , entity_id
            

            for i in range(1,len(ReadActions.intervals_ends)):
                #idx_group_actions=i
                interval = [list(ReadActions.intervals_ends.values())[i-1],list(ReadActions.intervals_ends.values())[i]]
                if(act_token  in ReadActions.intervals_dictRev and isInInterval(act_token , interval )) :
                    
                    _id = ReadActions.getEntityToken(lst_actions[k])
                    self.addToAudit(ReadActions.audit_col.index("ACTIONS"),i,act_token)
                    self.addToGroupAction(i , act_token)

        

                    if(list(ReadActions.intervals_ends.keys())[i] == "Times"):

                        if(ReadActions.intervals_dictRev.get(act_token) == "MOVE_TO"):
                            lst_mv = lst_actions[k][3]
                            self.extendColum('MOVE_TO',[lst_mv[0],len(lst_mv),lst_mv[-1]])
                            self.extendColum('MAPS',lst_mv)
                            
  
                            
                    elif(list(ReadActions.intervals_ends.keys())[i] == "Buffs"):
                        if(ReadActions.intervals_dictRev.get(act_token) == 'HEAL'):
                            self.addToColumn('LIFE',lst_actions[k][2])
                        elif(ReadActions.intervals_dictRev.get(act_token) in ['LOST_LIFE','NOVA_DAMAGE','DAMAGE_RETURN','LIFE_DAMAGE','POISON_DAMAGE',' AFTEREFFECT']):
                            self.addToColumn('LOST_LIFE',lst_actions[k][2])
                    elif(list(ReadActions.intervals_ends.keys())[i] == "Effects"):
                        if(ReadActions.intervals_dictRev.get(act_token) == 'ADD_WEAPON_EFFECT'):
                            self.addToColumn('WEAPON_DEG',lst_actions[k][5])
                            self.extendColum('WEAPON_ID',[_id])
                        elif(ReadActions.intervals_dictRev.get(act_token) == 'ADD_CHIP_EFFECT'):
                                self.addToColumn('CHIP_DEG',lst_actions[k][5])
                                self.extendColum('CHIP_ID',[_id])
                                
                    
                    
            k+=1
            idx+=1
        return k , entity_id

                            
                    
    def getFight ( self,lst_actions:list ):
        nbReads=ReadActions.whileToken("START_FIGHT",lst_actions)

        k=nbReads

        idx_turn = 0 
        while k < len(lst_actions):
            act_token =ReadActions.getCodeToken(lst_actions[k])
            if(self.isEndFight(act_token )):
                return k 
            nbReads , ent_id = self.getTurn(lst_actions[k:],idx_turn)
            if(nbReads==-1):
                break
            k+=(nbReads+1)
            if(k < len(lst_actions)):
                act_token =ReadActions.getCodeToken(lst_actions[k])
                if(self.isBeginTurn(act_token)):
                    idx_turn +=1


In [218]:
print(os.getcwd())

D:\Master\S2\TER\GitlabTER\AfterFight\src


In [219]:
json_file ="D:\Master\S2\TER\GitLabTER\AfterFight\outcome\243_10_12-62_1681001459881.json"
rr = ReadActions([12,62],243,10,json_file)

In [220]:

with open("../outcome/243_10_12-62_1681001459881.json", 'r') as f:
    data = json.loads(json.load(f))
    
lst_actions = data["fight"]["actions"]

In [221]:
rr.getFight (lst_actions)

In [222]:
display(rr.df)

,Times,Buffs,Funs,Effects,Others,LOST_LIFE,LIFE,MOVE_TO,WEAPON_DEG,CHIP_DEG,MAPS,WEAPON_ID,CHIP_ID,ID_GEN,ID_FIGHT,ID_ENTITY,ID_TURN,ACTIONS
0,"[[12, 12, 10, 12, 12, 12, 12, 13]]","[[101, 103]]",[[]],"[[302, 302, 302, 302, 302, 302, 302, 302, 302,...",[[]],[68],[43],"[486, 10, 499]",[0],[173],"[486, 503, 520, 502, 484, 466, 483, 465, 482, ...",[],"[14, 14, 96, 96, 19, 20, 20, 3, 21, 21]",243,10,62,0,"[[1, 4, 4, 1, 4, 4, 1, 1, 4, 2, 1, 4, 4, 1, 4,..."
1,"[[12, 12, 10, 12, 12, 12, 12, 13]]","[[101, 103]]",[[]],"[[302, 302, 302, 302, 302, 302, 302, 302, 302,...",[[]],[186],[43],"[581, 10, 463]",[0],[173],"[581, 564, 547, 530, 513, 496, 514, 497, 480, ...",[],"[14, 14, 96, 96, 19, 20, 20, 3, 21, 21]",243,10,12,0,"[[1, 4, 4, 1, 4, 4, 1, 1, 4, 2, 1, 4, 4, 1, 4,..."
2,"[[16, 16, 14, 13, 12, 12, 10, 10, 12]]","[[101, 101, 103, 103, 101]]",[[]],"[[301, 305, 301, 301, 305, 302, 305, 302, 302]]",[[]],[62],[22],"[482, 1, 482, 464, 2, 446]",[29],[16],"[482, 464, 446]","[41, 41, 41]","[4, 97, 19]",243,10,62,1,"[[1, 4, 2, 4, 4, 1, 4, 2, 2, 4, 1, 1, 1, 4, 2,..."
3,"[[16, 16, 14, 13, 12, 12, 12]]","[[110, 101, 103, 101, 103, 103, 103]]",[[]],"[[301, 305, 301, 301, 305, 305, 302, 302, 302]]",[[]],[86],[176],[],[29],[17],[],"[41, 41, 41]","[97, 4, 3]",243,10,12,1,"[[2, 1, 4, 2, 2, 4, 4, 1, 4, 2, 2, 4, 1, 1, 1,..."
4,"[[16, 16, 13, 12, 12, 12, 13, 10, 10, 10, 10, ...","[[110, 101, 101, 101, 101, 101, 101, 101, 101]]",[[]],"[[303, 303, 303, 301, 301, 301, 301, 301, 301,...",[[]],[19],[0],"[428, 1, 428, 446, 1, 446, 428, 1, 428, 446, 1...",[6],[15],"[428, 446, 428, 446, 428, 446, 428, 446]","[38, 38, 38, 38, 38, 38]","[97, 2, 18]",243,10,62,2,"[[2, 4, 4, 4, 1, 4, 2, 4, 2, 4, 2, 1, 4, 2, 4,..."
5,"[[16, 16, 13, 12, 12, 12, 13]]","[[110, 110, 101, 101, 101, 101, 101, 101, 101,...",[[]],"[[303, 303, 303, 301, 301, 301, 301, 301, 301,...",[[]],[34],[0],[],[6],[15],[],"[38, 38, 38, 38, 38, 38]","[97, 2, 18]",243,10,12,2,"[[2, 2, 4, 4, 4, 1, 4, 2, 4, 2, 4, 2, 1, 4, 2,..."
6,"[[16, 16, 13, 12, 12, 12, 10, 10, 10, 10, 10, ...","[[110, 110, 101, 103, 101, 103, 103, 103]]",[[]],"[[303, 303, 301, 301, 305, 302, 302, 302]]",[[]],[220],[114],"[428, 1, 428, 446, 1, 446, 428, 1, 428, 446, 1...",[2],[17],"[428, 446, 428, 446, 428, 446, 428, 446]","[45, 45]","[97, 4, 3]",243,10,62,3,"[[2, 2, 4, 4, 1, 4, 2, 2, 1, 4, 2, 2, 1, 1, 4,..."
7,"[[16, 16, 14, 13, 12, 12, 12]]","[[110, 110, 110, 101, 103, 101, 103, 103, 103]]",[[]],"[[303, 303, 301, 305, 301, 301, 305, 305, 302,...",[[]],[517],[249],[],[29],[17],[],"[41, 41, 41]","[97, 4, 3]",243,10,12,3,"[[2, 2, 2, 4, 4, 1, 4, 2, 2, 4, 4, 1, 4, 2, 2,..."
8,"[[16, 16, 13, 12, 12, 16, 10, 10, 10, 10, 10, ...","[[110, 110, 110, 101, 103, 101, 103, 101, 103,...",[[]],"[[303, 301, 301, 301, 301, 301, 301, 305, 302,...",[[]],[925],[172],"[428, 1, 428, 446, 1, 446, 428, 1, 428, 446, 1...",[9],[14],"[428, 446, 428, 446, 428, 446, 428, 446]","[38, 38, 38, 38, 38, 38, 38, 38, 38]","[97, 2]",243,10,62,4,"[[2, 2, 2, 4, 1, 4, 2, 2, 4, 2, 2, 4, 2, 2, 1,..."
9,"[[16, 16, 13, 12, 12, 16]]","[[110, 110, 110, 101, 103, 101, 103, 101, 103,...",[[]],"[[303, 303, 301, 301, 301, 301, 301, 301, 305,...",[[]],[760],[142],[],[9],[14],[],"[38, 38, 38, 38, 38, 38, 38, 38, 38]","[97, 2]",243,10,12,4,"[[2, 2, 2, 4, 4, 1, 4, 2, 2, 4, 2, 2, 4, 2, 2,..."
